In [2]:
import sbmlcore, pandas, numpy, copy
import itertools

pandas.options.display.max_columns=999
pandas.options.display.max_rows=150

## Read in three sets of clinical samples with mutations and recorded phenotypes to form the VALIDATION dataset

In [9]:
filestem = 'data/ds-validation'

validation_csvs = ['miotto2014','whitfield2015','cryptic2021']
stem='data/clinical-samples/ds-'
validation_dfs = {}
validation_sets = {}
for i in validation_csvs:
    validation_dfs[i] = pandas.read_csv(stem+i+'.csv')
    validation_dfs[i].set_index('MUTATION', inplace=True)
    validation_sets[i] = set(validation_dfs[i].index)
    validation_dfs[i].reset_index(inplace=True)
    validation_dfs[i].set_index(['MUTATION', 'IS_SNP', 'IN_CDS'], inplace=True)
    validation_dfs[i]['total'] = validation_dfs[i].R + validation_dfs[i].S

print("The number of mutations and samples in each set are:")
for i in validation_csvs:
    print(i, len(validation_sets[i]), validation_dfs[i].total.sum())

print("\n..and the intersection in numbers of mutations between sets are:")
if len(validation_csvs)>1:
    for i in list(itertools.combinations(validation_csvs,2)):
        print(i, len(validation_sets[i[0]].intersection(validation_sets[i[1]])))

print("\n..and the number of mutations in all three sets is:")
if len(validation_csvs)>2:
    for i in list(itertools.combinations(validation_csvs,3)):
        print(i, len(validation_sets[i[0]] & validation_sets[i[1]] & validation_sets[i[2]]))

The number of mutations and samples in each set are:
miotto2014 199 755
whitfield2015 65 634
cryptic2021 561 3578

..and the intersection in numbers of mutations between sets are:
('miotto2014', 'whitfield2015') 41
('miotto2014', 'cryptic2021') 179
('whitfield2015', 'cryptic2021') 47

..and the number of mutations in all three sets is:
('miotto2014', 'whitfield2015', 'cryptic2021') 35


Let's join them all and aggregate the AST results

In [10]:
VALIDATION = validation_dfs['cryptic2021'].join(validation_dfs['miotto2014'], lsuffix='s', rsuffix = 'm', how='outer')
VALIDATION = VALIDATION.join(validation_dfs['whitfield2015'], rsuffix='w',how='outer')
VALIDATION.rename(columns={'R': 'Rw', 'S': 'Sw', 'total': 'totalw'}, inplace=True)
VALIDATION.fillna(0, inplace=True)   
VALIDATION = VALIDATION.astype('int')
VALIDATION['R'] = VALIDATION['Rs'] + VALIDATION['Rm'] + VALIDATION['Rw']
VALIDATION['S'] = VALIDATION['Ss'] + VALIDATION['Sm'] + VALIDATION['Sw']
VALIDATION['TOTAL'] = VALIDATION['R'] + VALIDATION['S']
VALIDATION.drop(columns=['Rs', 'Ss', 'Rm', 'Sm', 'Rw', 'Sw', 'totals', 'totalm', 'totalw'], inplace=True)
VALIDATION.reset_index(inplace=True)
VALIDATION['PROP_R']=VALIDATION['R']/VALIDATION['TOTAL']
VALIDATION['PROP_S']=VALIDATION['S']/VALIDATION['TOTAL']
VALIDATION.columns.name='index'
VALIDATION

index,MUTATION,IS_SNP,IN_CDS,R,S,TOTAL,PROP_R,PROP_S
0,!187G,True,True,1,1,2,0.5,0.5
1,!187R,True,True,0,1,1,0.0,1.0
2,-29_indel,False,False,0,1,1,0.0,1.0
3,-2_indel,False,False,0,12,12,0.0,1.0
4,-32_indel,False,False,0,1,1,0.0,1.0
...,...,...,...,...,...,...,...,...
588,g-9a,True,False,0,1,1,0.0,1.0
589,t-10c,True,False,0,2,2,0.0,1.0
590,t-12c,True,False,9,1,10,0.9,0.1
591,t-7c,True,False,9,0,9,1.0,0.0


Apply the arbitrary rules described in the Methods to assign an overall phenotype 

In [11]:
def reliable_phenotype(row):
    reliable_phenotype=False
    phenotype='U'
    if row['TOTAL']>=4:
        if row['PROP_R']>=0.75:
            reliable_phenotype=True
            phenotype='R'
        elif row['PROP_S']>=0.75:
            reliable_phenotype=True
            phenotype='S'
    elif row['TOTAL']>=2:
        if row['R']==row['TOTAL']:
            reliable_phenotype=True
            phenotype='R'
        elif row['S']==row['TOTAL']:
            reliable_phenotype=True
            phenotype='S'

    return pandas.Series([reliable_phenotype, phenotype]) 

VALIDATION[['RELIABLE_PHENOTYPE', 'PHENOTYPE']]=VALIDATION.apply(reliable_phenotype,axis=1)   

print("%i samples cannot be assigned a phenotype" % ((~VALIDATION.RELIABLE_PHENOTYPE).sum()))

VALIDATION[VALIDATION.RELIABLE_PHENOTYPE].PHENOTYPE.value_counts()

307 samples cannot be assigned a phenotype


R    221
S     65
Name: PHENOTYPE, dtype: int64

Let's take a look at the mutations that cannot be assigned a phenotype using these rules

In [12]:
VALIDATION[~VALIDATION.RELIABLE_PHENOTYPE][:10]

index,MUTATION,IS_SNP,IN_CDS,R,S,TOTAL,PROP_R,PROP_S,RELIABLE_PHENOTYPE,PHENOTYPE
0,!187G,True,True,1,1,2,0.5,0.5,False,U
1,!187R,True,True,0,1,1,0.0,1.0,False,U
2,-29_indel,False,False,0,1,1,0.0,1.0,False,U
4,-32_indel,False,False,0,1,1,0.0,1.0,False,U
8,-9_indel,False,False,0,1,1,0.0,1.0,False,U
9,102_indel,False,True,1,0,1,1.0,0.0,False,U
10,108_indel,False,True,1,0,1,1.0,0.0,False,U
11,109_indel,False,True,1,0,1,1.0,0.0,False,U
12,116_indel,False,True,1,0,1,1.0,0.0,False,U
13,117_indel,False,True,1,0,1,1.0,0.0,False,U


Many of them only have a single measurement:

In [13]:
VALIDATION[~VALIDATION.RELIABLE_PHENOTYPE].TOTAL.value_counts().sort_index()

1     229
2      14
3      16
4       3
5       2
6       8
7       3
8       3
9       3
10      2
11      1
12      2
13      1
14      1
15      2
17      1
18      1
19      1
20      1
22      1
23      2
24      1
25      2
27      1
29      1
51      1
55      1
59      1
76      1
84      1
Name: TOTAL, dtype: int64

Looking at the mutations with more than one sample we find an ambigious distribution of R and S results suggesting perhaps these are close to the MGIT breakpoint.

In [111]:
VALIDATION[(~VALIDATION.RELIABLE_PHENOTYPE) & (VALIDATION.TOTAL>1)][:10]

index,MUTATION,IS_SNP,IN_CDS,R,S,TOTAL,PROP_R,PROP_S,RELIABLE_PHENOTYPE,PHENOTYPE
0,!187G,True,True,1,1,2,0.500000,0.500000,False,U
34,201_indel,False,True,3,4,7,0.428571,0.571429,False,U
46,257_indel,False,True,1,1,2,0.500000,0.500000,False,U
112,477_indel,False,True,2,1,3,0.666667,0.333333,False,U
116,48_indel,False,True,1,1,2,0.500000,0.500000,False,U
136,67_indel,False,True,2,1,3,0.666667,0.333333,False,U
145,A102P,True,True,8,4,12,0.666667,0.333333,False,U
148,A102V,True,True,12,13,25,0.480000,0.520000,False,U
150,A134G,True,True,1,1,2,0.500000,0.500000,False,U
162,A171E,True,True,2,2,4,0.500000,0.500000,False,U


Let's remove all these samples where a phenotype cannot be inferred using the arbitary rules

In [112]:
VALIDATION = VALIDATION[VALIDATION.RELIABLE_PHENOTYPE]

In [113]:
def classify_variant(row):
    is_cds=False
    is_snp=False
    is_nonsyn=False
    is_missense=False
    cols = row.MUTATION.split('_')
    if '-' not in row.MUTATION:
        is_cds=True        
        if len(cols)!=3 and 'indel' not in row.MUTATION:
            is_snp=True
            if row.MUTATION[0]!=row.MUTATION[-1]:
                is_nonsyn=True
                if row.MUTATION[-1]!='!':
                    is_missense=True
    else:
        if len(cols)!=3 and 'indel' not in row.MUTATION:
            is_snp=True
    
    return pandas.Series([is_cds,is_snp,is_nonsyn,is_missense])    

VALIDATION[['IN_CDS', 'IS_SNP', 'IS_NONSYN', 'IS_MISSENSE']] = VALIDATION.apply(classify_variant, axis=1)

pandas.crosstab(VALIDATION.IN_CDS,[VALIDATION.IS_SNP, VALIDATION.IS_NONSYN, VALIDATION.IS_MISSENSE])

/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_88397/858835743.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLINICAL_SAMPLES[['IN_CDS', 'IS_SNP', 'IS_NONSYN', 'IS_MISSENSE']] = CLINICAL_SAMPLES.apply(classify_variant, axis=1)
/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_88397/858835743.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLINICAL_SAMPLES[['IN_CDS', 'IS_SNP', 'IS_NONSYN', 'IS_MISSENSE']] = CLINICAL_SAMPLES.apply(classify_variant, axis=1)
/var/folders/84/9nb2

IS_SNP      False True             
IS_NONSYN   False False True       
IS_MISSENSE False False False True 
IN_CDS                             
False           4     8     0     0
True           45    12    13   195

Hence we find there are 12 promoter mutations (incl. 4 indels) and 45 indels in the CDS, with 12 synoymous mutations, 13 nonsense mutations and 195 missense mutations

In [122]:
print("Table 1: There are %i non-redundant missense mutations in this dataset" % (len(VALIDATION[ VALIDATION.IS_MISSENSE])))

Table 1: There are 195 non-redundant missense mutations in this dataset


But we cannot structurally model (i) mutations in the Stop codon and also (ii) mutations in resid 186 since it is not resolved in the protein structure so let's identify these

In [114]:
def valid_for_structure(row):
    if row.IN_CDS and row.IS_SNP and row.IS_NONSYN and row.IS_MISSENSE:
        if "!" in row.MUTATION:
            return False
        elif '186' in row.MUTATION:
            return False
        else:
            return True
    else:
        return False

VALIDATION['STRUCTURALLY_VALID'] = VALIDATION.apply(valid_for_structure, axis=1)        

/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_88397/2247709330.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CLINICAL_SAMPLES['STRUCTURALLY_VALID'] = CLINICAL_SAMPLES.apply(valid_for_structure, axis=1)


In [115]:
VALIDATION[ VALIDATION.IS_MISSENSE].PHENOTYPE.value_counts()

R    155
S     40
Name: PHENOTYPE, dtype: int64

But how many of these can we not map onto the protein structure?

In [116]:
VALIDATION[(VALIDATION.STRUCTURALLY_VALID) & (VALIDATION.IS_MISSENSE)].PHENOTYPE.value_counts(dropna=False)

R    155
S     40
Name: PHENOTYPE, dtype: int64

None are lost for this dataset, but let's subset down anyway

In [117]:
DATASET=VALIDATION[(VALIDATION.RELIABLE_PHENOTYPE) & (VALIDATION.STRUCTURALLY_VALID) & (VALIDATION.IS_MISSENSE)]

In [118]:
DATASET


index,MUTATION,IS_SNP,IN_CDS,R,S,TOTAL,PROP_R,PROP_S,RELIABLE_PHENOTYPE,PHENOTYPE,IS_NONSYN,IS_MISSENSE,STRUCTURALLY_VALID
146,A102R,True,True,0,3,3,0.000000,1.000000,True,S,True,True,True
147,A102T,True,True,3,0,3,1.000000,0.000000,True,R,True,True,True
151,A134V,True,True,23,2,25,0.920000,0.080000,True,R,True,True,True
152,A143G,True,True,6,0,6,1.000000,0.000000,True,R,True,True,True
154,A143T,True,True,1,6,7,0.142857,0.857143,True,S,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,W68S,True,True,2,0,2,1.000000,0.000000,True,R,True,True,True
539,Y103D,True,True,4,0,4,1.000000,0.000000,True,R,True,True,True
540,Y103H,True,True,5,0,5,1.000000,0.000000,True,R,True,True,True
541,Y103S,True,True,2,0,2,1.000000,0.000000,True,R,True,True,True


In [119]:
DATASET.rename(columns={'PHENOTYPE':'CONSISTENT_PHENOTYPE'}, inplace=True)
DATASET

/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_88397/953378862.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATASET.rename(columns={'PHENOTYPE':'CONSISTENT_PHENOTYPE'}, inplace=True)


index,MUTATION,IS_SNP,IN_CDS,R,S,TOTAL,PROP_R,PROP_S,RELIABLE_PHENOTYPE,CONSISTENT_PHENOTYPE,IS_NONSYN,IS_MISSENSE,STRUCTURALLY_VALID
146,A102R,True,True,0,3,3,0.000000,1.000000,True,S,True,True,True
147,A102T,True,True,3,0,3,1.000000,0.000000,True,R,True,True,True
151,A134V,True,True,23,2,25,0.920000,0.080000,True,R,True,True,True
152,A143G,True,True,6,0,6,1.000000,0.000000,True,R,True,True,True
154,A143T,True,True,1,6,7,0.142857,0.857143,True,S,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,W68S,True,True,2,0,2,1.000000,0.000000,True,R,True,True,True
539,Y103D,True,True,4,0,4,1.000000,0.000000,True,R,True,True,True
540,Y103H,True,True,5,0,5,1.000000,0.000000,True,R,True,True,True
541,Y103S,True,True,2,0,2,1.000000,0.000000,True,R,True,True,True


In [120]:
DATASET.loc[DATASET.CONSISTENT_PHENOTYPE.notna()].to_csv(filestem+'-full.csv',index=False)

DATASET.loc[DATASET.CONSISTENT_PHENOTYPE.notna()][['MUTATION','CONSISTENT_PHENOTYPE']].to_csv(filestem+'-phen.csv',index=False)

DATASET.loc[(DATASET.CONSISTENT_PHENOTYPE.notna())][['MUTATION']].to_csv(filestem+'-muts.csv',index=False, header=False)

DATASET['SEGID']='A'
DATASET.loc[(DATASET.CONSISTENT_PHENOTYPE.notna())][['SEGID','MUTATION']].to_csv(filestem+'-semu.csv',index=False, header=False, sep=' ')

DATASET.CONSISTENT_PHENOTYPE.value_counts(dropna=False)

/var/folders/84/9nb2gsqj4hlcxztvwf19wg680000gq/T/ipykernel_88397/692473263.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DATASET['SEGID']='A'


R    155
S     40
Name: CONSISTENT_PHENOTYPE, dtype: int64

In [123]:
DATASET

index,MUTATION,IS_SNP,IN_CDS,R,S,TOTAL,PROP_R,PROP_S,RELIABLE_PHENOTYPE,CONSISTENT_PHENOTYPE,IS_NONSYN,IS_MISSENSE,STRUCTURALLY_VALID,SEGID
146,A102R,True,True,0,3,3,0.000000,1.000000,True,S,True,True,True,A
147,A102T,True,True,3,0,3,1.000000,0.000000,True,R,True,True,True,A
151,A134V,True,True,23,2,25,0.920000,0.080000,True,R,True,True,True,A
152,A143G,True,True,6,0,6,1.000000,0.000000,True,R,True,True,True,A
154,A143T,True,True,1,6,7,0.142857,0.857143,True,S,True,True,True,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,W68S,True,True,2,0,2,1.000000,0.000000,True,R,True,True,True,A
539,Y103D,True,True,4,0,4,1.000000,0.000000,True,R,True,True,True,A
540,Y103H,True,True,5,0,5,1.000000,0.000000,True,R,True,True,True,A
541,Y103S,True,True,2,0,2,1.000000,0.000000,True,R,True,True,True,A


In [11]:
DST = pandas.read_csv('data/source-data/cryptic1-dst.csv')
MUTATIONS = pandas.read_csv('data/source-data/cryptic1-mutations.csv')

In [12]:
DST

,UNIQUEID,DRUG,SOURCE,METHOD_1,METHOD_2,METHOD_3,METHOD_CC,METHOD_MIC,PHENOTYPE
0,site.24.subj.PT-2.lab.1997-606.iso.1,PZA,SEQTREAT2020,liquid media,MGIT,BACTEC460,100.0,NaN,S
1,site.24.subj.PT-4.lab.1998-151.iso.1,PZA,SEQTREAT2020,liquid media,MGIT,BACTEC460,100.0,NaN,S
2,site.24.subj.PT-5.lab.1998-713.iso.1,PZA,SEQTREAT2020,liquid media,MGIT,BACTEC460,100.0,NaN,S
3,site.24.subj.PT-7.lab.1999-097.iso.1,PZA,SEQTREAT2020,liquid media,MGIT,BACTEC460,100.0,NaN,S
4,site.24.subj.PT-8.lab.1999-131.iso.1,PZA,SEQTREAT2020,liquid media,MGIT,BACTEC460,100.0,NaN,S
...,...,...,...,...,...,...,...,...,...
22837,site.00.subj.LE10KTB_21.lab.7627886.iso.1,PZA,NEJM2018,liquid media,MGIT,NaN,NaN,NaN,S
22838,site.00.subj.LE10KTB_8.lab.7627900.iso.1,PZA,NEJM2018,liquid media,MGIT,NaN,NaN,NaN,S
22839,site.00.subj.LE10KTB_12.lab.7628121.iso.1,PZA,NEJM2018,liquid media,MGIT,NaN,NaN,NaN,S
22840,site.00.subj.LE10KTB_14.lab.7628143.iso.1,PZA,NEJM2018,liquid media,MGIT,NaN,NaN,NaN,S


In [14]:
len(MUTATIONS.UNIQUEID.unique())

3549

In [15]:
len(MUTATIONS)

3573

In [20]:
len(MUTATIONS[MUTATIONS.AMINO_ACID_NUMBER.notna()].UNIQUEID.unique())

3331

In [23]:
foo = MUTATIONS[MUTATIONS.AMINO_ACID_NUMBER.notna()][['UNIQUEID', 'POSITION']].groupby('UNIQUEID').count()

In [25]:
foo.POSITION.value_counts()

1    3314
2      17
Name: POSITION, dtype: int64

In [29]:
singles = foo[foo.POSITION==1].index

In [32]:

MUTATIONS[(MUTATIONS.AMINO_ACID_NUMBER.notna()) & (MUTATIONS.UNIQUEID.isin(singles))].IS_INDEL.value_counts()

False    2911
True      403
Name: IS_INDEL, dtype: int64

In [43]:

indels = MUTATIONS[(MUTATIONS.AMINO_ACID_NUMBER.notna()) & (MUTATIONS.UNIQUEID.isin(singles)) & (MUTATIONS.IS_INDEL | MUTATIONS.IN_PROMOTER)].UNIQUEID.unique()


In [45]:
MUTATIONS[(MUTATIONS.AMINO_ACID_NUMBER.notna()) & (MUTATIONS.UNIQUEID.isin(singles)) & (MUTATIONS.IN_PROMOTER)].UNIQUEID.unique()

array([], dtype=object)

In [41]:
MUTATIONS

,UNIQUEID,GENE,MUTATION,POSITION,AMINO_ACID_NUMBER,GENOME_INDEX,NUCLEOTIDE_NUMBER,REF,ALT,IS_SNP,IS_INDEL,IN_CDS,IN_PROMOTER,IS_SYNONYMOUS,IS_NONSYNONYMOUS,IS_HET,IS_NULL,IS_FILTER_PASS,ELEMENT_TYPE,MUTATION_TYPE,INDEL_LENGTH,INDEL_1,INDEL_2,SITEID,NUMBER_NUCLEOTIDE_CHANGES
0,site.05.subj.PSLM-0791.lab.SLM-049.iso.1,pncA,L120P,120.0,120.0,NaN,NaN,ctg,ccg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,NaN,NaN,05,1
1,site.05.subj.LR-2032.lab.FN-00407-15.iso.1,pncA,W119L,119.0,119.0,NaN,NaN,tgg,ttg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,NaN,NaN,05,1
2,site.05.subj.PMK-1015.lab.MK-1781.iso.1,pncA,F58L,58.0,58.0,NaN,NaN,ttc,ctc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,NaN,NaN,05,1
3,site.05.subj.LR-2417.lab.FN-01304-17.iso.1,pncA,H51R,51.0,51.0,NaN,NaN,cac,cgc,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,NaN,NaN,05,1
4,site.05.subj.LR-2162.lab.FN-00284-16.iso.1,pncA,V139L,139.0,139.0,NaN,NaN,gtg,ctg,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,NaN,NaN,05,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3568,site.10.subj.YA00125758.lab.YA00125758.iso.1,pncA,467_indel,467.0,156.0,2288775.0,467.0,NaN,NaN,False,True,True,False,False,False,False,False,True,GENE,INDEL,1.0,467_ins,467_ins_1,10,0
3569,site.10.subj.SADG00497215_S8.lab.DG00497215_S8...,pncA,C138R,138.0,138.0,NaN,NaN,tgt,cgt,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,NaN,NaN,10,1
3570,site.10.subj.KD01666167.lab.KD01666167.iso.1,pncA,H71Y,71.0,71.0,NaN,NaN,cat,tat,True,False,True,False,False,True,False,False,True,GENE,AAM,NaN,NaN,NaN,10,1
3571,site.10.subj.YA00029870.lab.YA00029870.iso.1,pncA,a-11g,-11.0,NaN,2289252.0,-11.0,a,g,True,False,False,True,False,False,False,False,True,GENE,SNP,NaN,NaN,NaN,10,0


In [44]:

DST[DST.UNIQUEID.isin(indels)].PHENOTYPE.value_counts()

R    355
S     48
Name: PHENOTYPE, dtype: int64